In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        



def nearest_nonzero_idx(a,x,y):
    r,c = np.nonzero(a)
    min_idx = ((r - x)**2 + (c - y)**2).argmin()
    return r[min_idx], c[min_idx]


def emergence_pixels(gf, vel_x_raw, vel_y_raw, icethickness_raw, xres, yres, 
                     vel_min=0, max_velocity=600, vel_depth_avg_factor=0.8, option_border=1,
                     positive_is_east=True, positive_is_north=True, constant_icethickness=False, debug=True):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Glacier mask
    glac_mask = np.zeros(vel_x_raw.shape) + 1
    glac_mask[gf.z1.mask] = 0
    
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    if positive_is_north:
        vel_y = -1*vel_y_raw * vel_depth_avg_factor
    else:
        vel_y = vel_y_raw * vel_depth_avg_factor
    if positive_is_east:
        vel_x = vel_x_raw * vel_depth_avg_factor
    else:
        vel_x = -1*vel_x_raw * vel_depth_avg_factor
    vel_total = (vel_y**2 + vel_x**2)**0.5
    # Ice thickness
    icethickness = icethickness_raw.copy()
    if constant_icethickness:
        icethickness[:,:] = 1
        icethickness = icethickness * glac_mask
#     print('mean ice thickness:', np.round(icethickness.mean(),0), 'm')
    # Compute the initial volume
    volume_initial = icethickness * (xres * yres)
    pix_maxres = xres
    if yres > pix_maxres:
        pix_maxres = yres
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pix_maxres) + 1
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    vel_x[vel_total < vel_min] = 0
    vel_y[vel_total < vel_min] = 0
    vel_x[vel_total > max_velocity] = 0
    vel_y[vel_total > max_velocity] = 0
#     # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
#     if option_stagnantbands == 1:
#         vel_x[bands <= stagnant_band] = 0
#         vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / xres
    vel_y_pix = vel_y / yres
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
          
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
         
    # Redistribute off-glacier volume back onto the nearest pixel on the glacier
    offglac_row, offglac_col = np.where((glac_mask == 0) & (volume_final > 0))
    for nidx in range(0,len(offglac_row)):
        nrow = offglac_row[nidx]
        ncol = offglac_col[nidx]
        ridx, cidx = nearest_nonzero_idx(glac_mask, nrow, ncol)
        # Add off-glacier volume back onto nearest pixel on glacier
        volume_final[ridx,cidx] += volume_final[nrow,ncol]
        volume_final[nrow,ncol] = 0
            
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    if debug:
        print('Mass is conserved?', np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() < 0.01)
        print(np.round(np.absolute(volume_final.sum() - volume_initial.sum()),1), 
              np.round(np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() * 100,2), '%')
        
    if np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() > 0.01:
        print('MASS NOT CONSERVED FOR EMERGENCE VELOCITY')
    # Final ice thickness
    icethickness_final = volume_final / (xres * yres)
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity



class GlacFeat:
    def __init__(self, feat, glacname_fieldname, glacnum_fieldname):

        self.glacname = feat.GetField(glacname_fieldname)
        if self.glacname is None:
            self.glacname = ""
        else:
            #RGI has some nonstandard characters
            #self.glacname = self.glacname.decode('unicode_escape').encode('ascii','ignore')
            #glacname = re.sub(r'[^\x00-\x7f]',r'', glacname)
            self.glacname = re.sub(r'\W+', '', self.glacname)
            self.glacname = self.glacname.replace(" ", "")
            self.glacname = self.glacname.replace("_", "")
            self.glacname = self.glacname.replace("/", "")

        self.glacnum = feat.GetField(glacnum_fieldname)
        fn = feat.GetDefnRef().GetName()
        #RGIId (String) = RGI50-01.00004
        self.glacnum = '%0.5f' % float(self.glacnum.split('-')[-1])

        if self.glacname:
            self.feat_fn = "%s_%s" % (self.glacnum, self.glacname)
        else:
            self.feat_fn = str(self.glacnum)

        self.glac_geom_orig = geolib.geom_dup(feat.GetGeometryRef())
        self.glac_geom = geolib.geom_dup(self.glac_geom_orig)
        #Hack to deal with fact that this is not preserved in geom when loaded from pickle on disk
        self.glac_geom_srs_wkt = self.glac_geom.GetSpatialReference().ExportToWkt()

        #Attributes written by mb_calc
        self.z1 = None
        self.z1_hs = None
        self.z1_stats = None
        self.z1_ela = None
        self.z2 = None
        self.z2_hs = None
        self.z2_stats = None
        self.z2_ela = None
        self.z2_aspect = None
        self.z2_aspect_stats = None
        self.z2_slope = None
        self.z2_slope_stats = None
        self.res = None
        self.dhdt = None
        self.mb = None
        self.mb_mean = None
        self.t1 = None
        self.t2 = None
        self.dt = None
        self.t1_mean = None
        self.t2_mean = None
        self.dt_mean = None

        self.H = None
        self.H_mean = np.nan
        self.vx = None
        self.vy = None
        self.vm = None
        self.vm_mean = np.nan
        self.divQ = None
        self.emvel = None
        self.debris_class = None
        self.debris_thick = None
        self.debris_thick_mean = np.nan
        self.perc_clean = np.nan
        self.perc_debris = np.nan
        self.perc_pond = np.nan

    def geom_srs_update(self, srs=None):
        if self.glac_geom.GetSpatialReference() is None:
            if srs is None:
                srs = osr.SpatialReference()
                srs.ImportFromWkt(self.glac_geom_srs_wkt)
            self.glac_geom.AssignSpatialReference(srs)

    def geom_attributes(self, srs=None):
        self.geom_srs_update()
        if srs is not None:
            #Should reproject here to equal area, before geom_attributes
            #self.glac_geom.AssignSpatialReference(glac_shp_srs)
            #self.glac_geom_local = geolib.geom2localortho(self.glac_geom)
            geolib.geom_transform(self.glac_geom, srs)

        self.glac_geom_extent = geolib.geom_extent(self.glac_geom)
        self.glac_area = self.glac_geom.GetArea()
        self.glac_area_km2 = self.glac_area / 1E6
        self.cx, self.cy = self.glac_geom.Centroid().GetPoint_2D()
        
        
#RGI uses 50 m bins
def hist_plot(gf, bin_width=50.0, dz_clim=(-2.0, 2.0), exportcsv=True, csv_ending=''):
    #print("Generating histograms")
    #Create bins for full range of input data and specified bin width

    #NOTE: these counts/areas are for valid pixels only
    #Not necessarily a true representation of actual glacier hypsometry
    #Need a void-filled DEM for this

    z_bin_edges, z_bin_centers = malib.get_bins(gf.z1, bin_width)
    #Need to compress here, otherwise histogram uses masked values!
    z1_bin_counts, z1_bin_edges = np.histogram(gf.z1.compressed(), bins=z_bin_edges)
    z1_bin_areas = z1_bin_counts * gf.res[0] * gf.res[1] / 1E6
    #RGI standard is integer thousandths of glaciers total area
    #Should check to make sure sum of bin areas equals total area
    #z1_bin_areas_perc = 100. * z1_bin_areas / np.sum(z1_bin_areas)
    z1_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)

    #If we only have one elevation grid with dhdt
    if gf.z2 is not None:
        z2_bin_counts, z2_bin_edges = np.histogram(gf.z2.compressed(), bins=z_bin_edges)
        z2_bin_areas = z2_bin_counts * gf.res[0] * gf.res[1] / 1E6
        #z2_bin_areas_perc = 100. * z2_bin_areas / np.sum(z2_bin_areas)
        z2_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)
    else:
        z2_bin_counts = z1_bin_counts
        z2_bin_edges = z1_bin_edges
        z2_bin_areas = z1_bin_areas
        z2_bin_areas_perc = z1_bin_areas_perc

    #Create arrays to store output
    slope_bin_med = np.ma.masked_all_like(z1_bin_areas)
    slope_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    aspect_bin_med = np.ma.masked_all_like(z1_bin_areas)
    aspect_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.dhdt is not None:
        mb_bin_med = np.ma.masked_all_like(z1_bin_areas)
        np.ma.set_fill_value(mb_bin_med, np.nan)
        mb_bin_mad = np.ma.masked_all_like(mb_bin_med)
        mb_bin_mean = np.ma.masked_all_like(mb_bin_med)
        mb_bin_std = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_med = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_mad = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_mean = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_std = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_count = np.ma.masked_all_like(mb_bin_med)
    if gf.vm is not None:
        vm_bin_med = np.ma.masked_all_like(z1_bin_areas)
        vm_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.H is not None:
        H_bin_mean = np.ma.masked_all_like(z1_bin_areas)
        H_bin_std = np.ma.masked_all_like(z1_bin_areas)
    if gf.emvel is not None:
        emvel_bin_mean = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_std = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_med = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.debris_class is not None:
#         perc_clean = np.ma.masked_all_like(z1_bin_areas)
#         perc_debris = np.ma.masked_all_like(z1_bin_areas)
#         perc_pond = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_med = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_mad = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_clean_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_debris_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_pond_bin_med = np.ma.masked_all_like(mz1_bin_areas)

#         gf.dhdt_clean = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 1).data))
#         gf.dhdt_debris = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 2).data))
#         gf.dhdt_pond = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 3).data))

    if gf.debris_thick_ts is not None:
        debris_thick_ts_med = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_ts_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.meltfactor_ts is not None:
        meltfactor_ts_med = np.ma.masked_all_like(z1_bin_areas)
        meltfactor_ts_mad = np.ma.masked_all_like(z1_bin_areas)

    #Bin sample count must be greater than this value
    min_bin_samp_count = 9

    #Loop through each bin and extract stats
    idx = np.digitize(gf.z1, z_bin_edges)
    for bin_n in range(z_bin_centers.size):
        if gf.dhdt is not None:
            mb_bin_samp = gf.mb_map[(idx == bin_n+1)]
            if mb_bin_samp.count() > min_bin_samp_count:
                mb_bin_med[bin_n] = malib.fast_median(mb_bin_samp)
                mb_bin_mad[bin_n] = malib.mad(mb_bin_samp)
                mb_bin_mean[bin_n] = mb_bin_samp.mean()
                mb_bin_std[bin_n] = mb_bin_samp.std()
            dhdt_bin_samp = gf.dhdt[(idx == bin_n+1)]
            if dhdt_bin_samp.count() > min_bin_samp_count:
                dhdt_bin_med[bin_n] = malib.fast_median(dhdt_bin_samp)
                dhdt_bin_mad[bin_n] = malib.mad(dhdt_bin_samp)
                dhdt_bin_mean[bin_n] = dhdt_bin_samp.mean()
                dhdt_bin_std[bin_n] = dhdt_bin_samp.std()
                dhdt_bin_count[bin_n] = dhdt_bin_samp.count()
        if gf.debris_thick is not None:
            debris_thick_bin_samp = gf.debris_thick[(idx == bin_n+1)]
            if debris_thick_bin_samp.size > min_bin_samp_count:
                debris_thick_med[bin_n] = malib.fast_median(debris_thick_bin_samp)
                debris_thick_mad[bin_n] = malib.mad(debris_thick_bin_samp)
        
        if gf.debris_thick_ts is not None:
            debris_thick_ts_bin_samp = gf.debris_thick_ts[(idx == bin_n+1)]
            if debris_thick_ts_bin_samp.size > min_bin_samp_count:
                debris_thick_ts_med[bin_n] = malib.fast_median(debris_thick_ts_bin_samp)
                debris_thick_ts_mad[bin_n] = malib.mad(debris_thick_ts_bin_samp)
        if gf.meltfactor_ts is not None:
            meltfactor_ts_bin_samp = gf.meltfactor_ts[(idx == bin_n+1)]
            if meltfactor_ts_bin_samp.size > min_bin_samp_count:
                meltfactor_ts_med[bin_n] = malib.fast_median(meltfactor_ts_bin_samp)
                meltfactor_ts_mad[bin_n] = malib.mad(meltfactor_ts_bin_samp)
                
        if gf.debris_class is not None:
            debris_class_bin_samp = gf.debris_class[(idx == bin_n+1)]
            dhdt_clean_bin_samp = gf.dhdt_clean[(idx == bin_n+1)]
            dhdt_debris_bin_samp = gf.dhdt_debris[(idx == bin_n+1)]
            dhdt_pond_bin_samp = gf.dhdt_pond[(idx == bin_n+1)]
            if debris_class_bin_samp.count() > min_bin_samp_count:
                perc_clean[bin_n] = 100. * (debris_class_bin_samp == 1).sum()/debris_class_bin_samp.count()
                perc_debris[bin_n] = 100. * (debris_class_bin_samp == 2).sum()/debris_class_bin_samp.count()
                perc_pond[bin_n] = 100. * (debris_class_bin_samp == 3).sum()/debris_class_bin_samp.count()
            if dhdt_clean_bin_samp.count() > min_bin_samp_count:
                dhdt_clean_bin_med[bin_n] = malib.fast_median(dhdt_clean_bin_samp)
            if dhdt_debris_bin_samp.count() > min_bin_samp_count:
                dhdt_debris_bin_med[bin_n] = malib.fast_median(dhdt_debris_bin_samp)
            if dhdt_pond_bin_samp.count() > min_bin_samp_count:
                dhdt_pond_bin_med[bin_n] = malib.fast_median(dhdt_pond_bin_samp)
        if gf.vm is not None:
            vm_bin_samp = gf.vm[(idx == bin_n+1)]
            if vm_bin_samp.size > min_bin_samp_count:
                vm_bin_med[bin_n] = malib.fast_median(vm_bin_samp)
                vm_bin_mad[bin_n] = malib.mad(vm_bin_samp)
        if gf.H is not None:
            H_bin_samp = gf.H[(idx == bin_n+1)]
            if H_bin_samp.size > min_bin_samp_count:
                H_bin_mean[bin_n] = H_bin_samp.mean()
                H_bin_std[bin_n] = H_bin_samp.std()
        if gf.emvel is not None:
            emvel_bin_samp = gf.emvel[(idx == bin_n+1)]
            if emvel_bin_samp.size > min_bin_samp_count:
                emvel_bin_mean[bin_n] = emvel_bin_samp.mean()
                emvel_bin_std[bin_n] = emvel_bin_samp.std()
                emvel_bin_med[bin_n] = malib.fast_median(emvel_bin_samp)
                emvel_bin_mad[bin_n] = malib.mad(emvel_bin_samp)
        slope_bin_samp = gf.z1_slope[(idx == bin_n+1)]
        if slope_bin_samp.size > min_bin_samp_count:
            slope_bin_med[bin_n] = malib.fast_median(slope_bin_samp)
            slope_bin_mad[bin_n] = malib.mad(slope_bin_samp)
        aspect_bin_samp = gf.z1_aspect[(idx == bin_n+1)]
        if aspect_bin_samp.size > min_bin_samp_count:
            aspect_bin_med[bin_n] = malib.fast_median(aspect_bin_samp)
            aspect_bin_mad[bin_n] = malib.mad(aspect_bin_samp)

    if gf.dhdt is not None:
        dhdt_bin_areas = dhdt_bin_count * gf.res[0] * gf.res[1] / 1E6
        #dhdt_bin_areas_perc = 100. * dhdt_bin_areas / np.sum(dhdt_bin_areas)
        dhdt_bin_areas_perc = 100. * (dhdt_bin_areas / gf.glac_area_km2)

    outbins_header = 'bin_center_elev_m, z1_bin_count_valid, z1_bin_area_valid_km2, z1_bin_area_perc, z2_bin_count_valid, z2_bin_area_valid_km2, z2_bin_area_perc, slope_bin_med, aspect_bin_med'
    fmt = '%0.1f, %0.0f, %0.3f, %0.2f, %0.0f, %0.3f, %0.2f, %0.2f, %0.2f'
    outbins = [z_bin_centers, z1_bin_counts, z1_bin_areas, z1_bin_areas_perc, z2_bin_counts, z2_bin_areas, z2_bin_areas_perc, slope_bin_med, aspect_bin_med]
    if gf.dhdt is not None:
        outbins_header += ', dhdt_bin_count, dhdt_bin_area_valid_km2, dhdt_bin_area_perc, dhdt_bin_med_ma, dhdt_bin_mad_ma, dhdt_bin_mean_ma, dhdt_bin_std_ma, mb_bin_med_mwea, mb_bin_mad_mwea, mb_bin_mean_mwea, mb_bin_std_mwea'
        fmt += ', %0.0f, %0.3f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
        outbins.extend([dhdt_bin_count, dhdt_bin_areas, dhdt_bin_areas_perc, dhdt_bin_med, dhdt_bin_mad, dhdt_bin_mean, dhdt_bin_std, \
                        mb_bin_med, mb_bin_mad, mb_bin_mean, mb_bin_std])
    if gf.debris_thick is not None:
        outbins_header += ', debris_thick_med_m, debris_thick_mad_m'
        fmt += ', %0.2f, %0.2f'
        debris_thick_med[debris_thick_med == -(np.inf)] = 0.00
        debris_thick_mad[debris_thick_mad == -(np.inf)] = 0.00
        outbins.extend([debris_thick_med, debris_thick_mad])
    
    if gf.debris_thick_ts is not None:
        outbins_header += ',debris_thick_ts_med_m,debris_thick_ts_mad_m'
        fmt += ', %0.2f, %0.2f'
        debris_thick_ts_med[debris_thick_ts_med == -(np.inf)] = 0.00
        debris_thick_ts_mad[debris_thick_ts_mad == -(np.inf)] = 0.00
        outbins.extend([debris_thick_ts_med, debris_thick_ts_mad])
    if gf.meltfactor_ts is not None:
        outbins_header += ',meltfactor_ts_med_m,meltfactor_ts_mad_m'
        fmt += ', %0.2f, %0.2f'
        meltfactor_ts_med[meltfactor_ts_med == -(np.inf)] = 1
        meltfactor_ts_med[meltfactor_ts_med > 1] = 1
        meltfactor_ts_med[meltfactor_ts_med <= 0] = 1
        meltfactor_ts_mad[meltfactor_ts_mad == -(np.inf)] = 0
        meltfactor_ts_mad[meltfactor_ts_mad > 1] = 0
        meltfactor_ts_mad[meltfactor_ts_mad <= 0] = 0
        outbins.extend([meltfactor_ts_med, meltfactor_ts_mad])
    
    if gf.debris_class is not None:
        outbins_header += ', perc_debris, perc_pond, perc_clean, dhdt_debris_med, dhdt_pond_med, dhdt_clean_med'
        fmt += ', %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
        outbins.extend([perc_debris, perc_pond, perc_clean, dhdt_debris_bin_med, dhdt_pond_bin_med, dhdt_clean_bin_med])
    if gf.vm is not None:
        outbins_header += ', vm_med, vm_mad'
        fmt += ', %0.2f, %0.2f'
        outbins.extend([vm_bin_med, vm_bin_mad])
    if gf.H is not None:
        outbins_header += ', H_mean, H_std'
        fmt += ', %0.2f, %0.2f'
        outbins.extend([H_bin_mean, H_bin_std])
#         outbins_header += ', H_mean, H_std, emvel_mean, emvel_std'
#         fmt += ', %0.2f, %0.2f, %0.2f, %0.2f'
#         outbins.extend([H_bin_mean, H_bin_std, emvel_bin_mean, emvel_bin_std])

    if gf.emvel is not None:
        outbins_header += ', emvel_mean, emvel_std, emvel_med, emvel_mad'
        fmt += ', %0.3f, %0.3f, %0.3f, %0.3f'
        outbins.extend([emvel_bin_mean, emvel_bin_std, emvel_bin_med, emvel_bin_mad])

    outbins = np.ma.array(outbins).T.astype('float32')
    np.ma.set_fill_value(outbins, np.nan)
    outbins = outbins.filled(np.nan)
    if exportcsv:
        outbins_fn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
        np.savetxt(outbins_fn, outbins, fmt=fmt, delimiter=',', header=outbins_header)
    
    outbins_df = pd.DataFrame(outbins, columns=outbins_header.split(','))
    return outbins_df, z_bin_edges
#     return z_bin_edges

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib
# from imview.lib import pltlib

In [3]:
import globaldebris_input as input

#INPUT
# topdir='/Users/davidrounce/Documents/Dave_Rounce/HiMAT/DEMs/'
# #Output directory
# outdir = topdir + 'Shean_2019_0213/mb_combined_20190213_nmad_bins/'
# outdir_fig = outdir + '/figures/'
# outdir_csv = outdir + '/csv'

verbose=False
extra_layers=True
min_glac_area_writeout=0
min_valid_area_perc = 0
buff_dist = 1000
bin_width = 5

ts_info_fullfn = input.ts_fp + input.roi + '_debris_tsinfo.nc'

print(ts_info_fullfn)

#INPUT
glac_shp_fn_dict = {'13':input.main_directory + '/../../../HiMAT/RGI/rgi60/13_rgi60_CentralAsia/13_rgi60_CentralAsia.shp',
                    '14':input.main_directory + '/../../../HiMAT/RGI/rgi60/14_rgi60_SouthAsiaWest/14_rgi60_SouthAsiaWest.shp',
                    '15':input.main_directory + '/../../../HiMAT/RGI/rgi60/15_rgi60_SouthAsiaEast/15_rgi60_SouthAsiaEast.shp'}
glac_shp_proj_fp = input.output_fp + 'glac_shp_proj/'
if os.path.exists(glac_shp_proj_fp) == False:
    os.makedirs(glac_shp_proj_fp)

#DEM
z1_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/surface_DEMs_RGI60/' + 
          'surface_DEMs_RGI60-XXXX/')
z1_fn_sample = 'surface_DEM_RGI60-XXXX.tif'
# Ice thickness
huss_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/' + 
                   'composite_thickness_RGI60-all_regions/RGI60-XXXX/')
huss_fn_sample = 'RGI60-XXXX_thickness.tif'

if os.path.exists(input.ts_fp) == False:
    os.makedirs(input.ts_fp)
    
outdir_csv = input.outdir_emvel_fp 
outdir_fig = input.outdir_emvel_fp  + '../figures/'

if os.path.exists(glac_shp_proj_fp) == False:
    os.makedirs(glac_shp_proj_fp)
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)
if os.path.exists(outdir_fig) == False:
    os.makedirs(outdir_fig)



/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/01_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(input.mb_binned_fp):
    if i.endswith('mb_bins_wdc_emvel_offset.csv'):
        region = int(i.split('.')[0])
        if region in input.roi_rgidict[input.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        
        
rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)

print(len(rgiid_list))

main_glac_rgi = input.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

791
791 glaciers in region 1 are included in this model run: ['00013', '00021', '00037', '00041', '00042', '00556', '00570', '00571', '00660', '00670', '00675', '00688', '00703', '00704', '00709', '00732', '00739', '00746', '00780', '00787', '00792', '00799', '00814', '00852', '00865', '00870', '00871', '00903', '00942', '00951', '00960', '00962', '00982', '01050', '01077', '01104', '01150', '01153', '01162', '01182', '01223', '01242', '01268', '01275', '01276', '01277', '01282', '01284', '01292', '01306'] and more
This study is focusing on 791 glaciers in region [1]


In [5]:
# Merge with debris cover stats
dc_shp = gpd.read_file(input.debriscover_fp + input.debriscover_fn_dict[input.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

# main_glac_rgi['DC_Area_%'] = 0
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)

In [6]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(input.ostrem_fp):
    if i.endswith(input.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(input.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi.index.values[6855]]):

#     if verbose:
#         print(nglac, glac_idx, main_glac_rgi.loc[glac_idx,'rgino_str'], 
#               main_glac_rgi.loc[glac_idx,'CenLat'], main_glac_rgi.loc[glac_idx,'CenLon'])
        
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,12,RGI60-01.00013,-146.684082,63.499329,1,2,209.630,823,4003,1848,...,20090703,13,01.00013,1.00013,213.315918,1.00013_mb_bins_wdc_emvel_offset.csv,17.84,63.50,213.25,6350N-21325E_debris_melt_curve.nc
1,20,RGI60-01.00021,-146.606262,63.404831,1,2,56.531,942,2584,1781,...,20090703,21,01.00021,1.00021,213.393738,1.00021_mb_bins_wdc_emvel_offset.csv,1.82,63.50,213.50,6350N-21350E_debris_melt_curve.nc
2,36,RGI60-01.00037,-146.528168,63.469173,1,2,234.583,722,3081,1841,...,20090703,37,01.00037,1.00037,213.471832,1.00037_mb_bins_wdc_emvel_offset.csv,13.54,63.50,213.50,6350N-21350E_debris_melt_curve.nc
3,40,RGI60-01.00041,-147.107000,63.657000,1,2,90.905,948,3734,1705,...,20090703,41,01.00041,1.00041,212.893000,1.00041_mb_bins_wdc_emvel_offset.csv,22.97,63.75,213.00,6375N-21300E_debris_melt_curve.nc
4,41,RGI60-01.00042,-147.579269,63.594097,1,2,83.656,830,3734,1578,...,20090703,42,01.00042,1.00042,212.420731,1.00042_mb_bins_wdc_emvel_offset.csv,15.06,63.50,212.50,6350N-21250E_debris_melt_curve.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,26733,RGI60-01.26738,-140.388550,60.879704,1,5,718.416,792,5241,2416,...,20050812,26738,01.26738,1.26738,219.611450,1.26738_mb_bins_wdc_emvel_offset.csv,15.40,61.00,219.50,6100N-21950E_debris_melt_curve.nc
787,26738,RGI60-01.26743,-151.443481,62.802460,1,2,176.235,488,3740,1295,...,20100912,26743,01.26743,1.26743,208.556519,1.26743_mb_bins_wdc_emvel_offset.csv,28.09,62.75,208.50,6275N-20850E_debris_melt_curve.nc
788,26856,RGI60-01.26861,-134.732000,59.454000,1,6,3.545,1404,1879,1506,...,20050811,26861,01.26861,1.26861,225.268000,1.26861_mb_bins_wdc_emvel_offset.csv,12.14,59.50,225.25,5950N-22525E_debris_melt_curve.nc


In [7]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[0]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
#     lat_idx = np.where(abs(main_glac_rgi_subset.loc[0,'lat_nearest'] - lat_values) ==
#                        abs(main_glac_rgi_subset.loc[0,'lat_nearest'] - lat_values).min())[0][0]
#     lon_idx = np.where(abs(main_glac_rgi_subset.loc[0,'lon_nearest'] - lon_values) == 
#                        abs(main_glac_rgi_subset.loc[0,'lon_nearest'] - lon_values).min())[0][0]
    
    print(nlatlon, lat_values[lat_idx], lon_values[lon_idx], ostrem_fn)

    df_all = None
    
    doy_list = []
    year_list = []
    dayfrac_list = []
        
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if verbose:
            print(nglac, glac_idx, rgiid,'\n')
        
        # Process debris-covered glaciers
        if main_glac_rgi.loc[glac_idx,'DC_Area_%'] > input.dc_percarea_threshold:

#             if verbose:
            print('  processing', glac_str)

            # ===== Project shapefile =====
            huss_dir = huss_dir_sample.replace('XXXX',str(region.zfill(2)))
            huss_fn = huss_fn_sample.replace('XXXX',glac_str)

            proj_fn = os.path.join(huss_dir, huss_fn) # THIS PROJECTION IS KEY!
            ds = gdal.Open(proj_fn)
            prj = ds.GetProjection()
            srs = osr.SpatialReference(wkt=prj)
            aea_srs = srs

            # Shape layer processing
            glac_shp_init = gpd.read_file(input.glac_shp_fn_dict[region])
            if verbose:
                print('Shp init crs:', glac_shp_init.crs)

            glac_shp_single = glac_shp_init[glac_shp_init['RGIId'] == rgiid]
            glac_shp_single = glac_shp_single.reset_index()
            
            # If projected shapefile already exists, then skip projection
            glac_shp_proj_fn = glac_shp_proj_fp + glac_str + '_crs' + str(aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp'

            if os.path.exists(glac_shp_proj_fn) == False:
                glac_shp_proj = glac_shp_single.to_crs({'init': 'epsg:' + str(aea_srs.GetAttrValue("AUTHORITY", 1))})
                glac_shp_proj.to_file(glac_shp_proj_fn)

            glac_shp_ds = ogr.Open(glac_shp_proj_fn, 0)
            glac_shp_lyr = glac_shp_ds.GetLayer()
            #This should be contained in features
            glac_shp_srs = glac_shp_lyr.GetSpatialRef()
            feat_count = glac_shp_lyr.GetFeatureCount()
            if verbose:
                print("Input glacier polygon count: %i" % feat_count)

            # Load DEM
            z1_dir = z1_dir_sample.replace('XXXX',str(region.zfill(2)))
            z1_fn = z1_fn_sample.replace('XXXX',glac_str)
            z1_ds = gdal.Open(z1_dir + z1_fn)
            z1_int_geom = geolib.ds_geom_intersection([z1_ds, z1_ds], t_srs=glac_shp_srs)

            glacfeat_list = []
            glacname_fieldname = "Name"
            glacnum_fieldname = "RGIId"
            glacnum_fmt = '%08.5f'

            for n, feat in enumerate(glac_shp_lyr):
                gf = GlacFeat(feat, glacname_fieldname, glacnum_fieldname)
                if verbose:
                    print("%i of %i: %s" % (n+1, feat_count, gf.feat_fn))
                #NOTE: Input must be in projected coordinate system, ideally equal area
                #Should check this and reproject
                gf.geom_attributes(srs=aea_srs)
                glacfeat_list.append(gf)

            if verbose:
                print(gf.feat_fn)

            fn_dict = OrderedDict()
            #We at least want to warp the two input DEMs
            fn_dict['z1'] = os.path.join(z1_dir, z1_fn)

            if extra_layers and (gf.glac_area_km2 > min_glac_area_writeout):
                if verbose:
                    print(gf.glacnum)

                # Ice thickness data
                ice_thick_fn = os.path.join(huss_dir, huss_fn)
                if os.path.exists(ice_thick_fn):
                    fn_dict['ice_thick'] = ice_thick_fn


                if os.path.exists(input.ts_fp + input.ts_fn_dict[input.roi]):
                    fn_dict['ts'] = input.ts_fp + input.ts_fn_dict[input.roi]
                
                if os.path.exists(input.ts_fp + input.ts_dayfrac_fn_dict[input.roi]):
                    fn_dict['ts_dayfrac'] = input.ts_fp + input.ts_dayfrac_fn_dict[input.roi]
                if os.path.exists(input.ts_fp + input.ts_year_fn_dict[input.roi]):
                    fn_dict['ts_year'] = input.ts_fp + input.ts_year_fn_dict[input.roi]
                if os.path.exists(input.ts_fp + input.ts_doy_fn_dict[input.roi]):
                    fn_dict['ts_doy'] = input.ts_fp + input.ts_doy_fn_dict[input.roi]


            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=buff_dist)
            if verbose:
                print("Expanding extent")
                print(gf.glac_geom_extent)
                print(warp_extent)
                print(aea_srs)

            #Warp everything to common res/extent/proj
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=input.ts_stats_res, \
                    extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
                    r='cubic')

            ds_dict = dict(zip(fn_dict.keys(), ds_list))

            if verbose:
                print(ds_list)
                print(fn_dict.keys())

            #Prepare mask for all glaciers within buffered area, not just the current glacier polygon
            glac_shp_ds = ogr.Open(glac_shp_proj_fn, 0)
            glac_shp_lyr = glac_shp_ds.GetLayer()

            #Get global glacier mask
            #Want this to be True over ALL glacier surfaces, not just the current polygon
            glac_shp_lyr_mask = geolib.lyr2mask(glac_shp_lyr, ds_dict['ice_thick'])

            #Create buffer around glacier polygon
            glac_geom_buff = gf.glac_geom.Buffer(buff_dist)
            #This is False over glacier polygon surface, True elsewhere - can be applied directly
            glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])

            # ds masks
            ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
            dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
            dems_mask = dem1.mask
            if verbose:
                print('list of datasets:', len(ds_list_masked), fn_dict.values())

            #Combine to identify ~1 km buffer around glacier polygon over static rock
            static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
            static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']))

                #Now apply glacier mask AND mask NaN values
                glac_geom_mask = np.ma.mask_or(glac_geom_mask, dems_mask)
                gf.z1 = np.ma.array(gf.z1, mask=glac_geom_mask)

                if verbose:
                    print('\n\n# z1 pixels:', gf.z1.count(), '\n')
                if gf.z1.count() == 0:
                    if verbose:
                        print("No z1 pixels")
            else:
                print("Unable to load z1 ds")

            # ===== ADD VARIOUS LAYERS TO gf =====
            if nlatlon + nglac == 0:
                print('\n\nHACK TO BYPASS VALID AREA\n\n')
            gf.valid_area_perc = 100

            if gf.valid_area_perc < (100. * min_valid_area_perc):
                if verbose:
                    print("Not enough valid pixels. %0.1f%% percent of glacier polygon area" % (gf.valid_area_perc))
            #     return None

            else:
                #Filter dz - throw out abs differences >150 m

                #Compute dz, volume change, mass balance and stats
                gf.z1_stats = malib.get_stats(gf.z1)
                z1_elev_med = gf.z1_stats[5]
                z1_elev_min, z1_elev_max = malib.calcperc(gf.z1, (0.1, 99.9))

                #Caluclate stats for aspect and slope using z2
                #Requires GDAL 2.1+
                gf.z1_aspect = np.ma.array(geolib.gdaldem_mem_ds(ds_dict['z1'], processing='aspect', returnma=True), mask=glac_geom_mask)
                gf.z1_aspect_stats = malib.get_stats(gf.z1_aspect)
                z1_aspect_med = gf.z1_aspect_stats[5]
                gf.z1_slope = np.ma.array(geolib.gdaldem_mem_ds(ds_dict['z1'], processing='slope', returnma=True), mask=glac_geom_mask)
                gf.z1_slope_stats = malib.get_stats(gf.z1_slope)
                z1_slope_med = gf.z1_slope_stats[5]

                #Can estimate ELA values computed from hypsometry and typical AAR
                #For now, assume ELA is mean
                gf.z1_ela = None
                gf.z1_ela = gf.z1_stats[3]
                #Note: in theory, the ELA should get higher with mass loss
                #In practice, using mean and same polygon, ELA gets lower as glacier surface thins


                if extra_layers and (gf.glac_area_km2 > min_glac_area_writeout):
                    if 'ice_thick' in ds_dict:
                        #Load ice thickness
                        gf.H = np.ma.array(iolib.ds_getma(ds_dict['ice_thick']), mask=glac_geom_mask)
                        gf.H_mean = gf.H.mean()
                        if verbose:
                            print('mean ice thickness [m]:', gf.H_mean)

                    if 'ts' in ds_dict:
                        #Load surface temperature maps
                        gf.ts = np.ma.array(iolib.ds_getma(ds_dict['ts']), mask=glac_geom_mask)
                        gf.ts.mask = np.ma.mask_or(glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    else:
                        gf.ts = None
                        
                    if 'ts_dayfrac' in ds_dict:
                        #Load surface temperature maps
                        gf.ts_dayfrac = np.ma.array(iolib.ds_getma(ds_dict['ts_dayfrac']), mask=glac_geom_mask)
                        gf.ts_dayfrac.mask = np.ma.mask_or(glac_geom_mask, 
                                                           np.ma.getmask(np.ma.masked_array(gf.ts_dayfrac.data, np.isnan(gf.ts_dayfrac.data))))
                    else:
                        gf.ts_dayfrac = None
                        
                    if 'ts_year' in ds_dict:
                        #Load surface temperature maps
                        gf.ts_year = np.ma.array(iolib.ds_getma(ds_dict['ts_year']), mask=glac_geom_mask)
                        gf.ts_year.mask = np.ma.mask_or(glac_geom_mask, 
                                                        np.ma.getmask(np.ma.masked_array(gf.ts_year.data, np.isnan(gf.ts_year.data))))
                    else:
                        gf.ts_year = None
                        
                    if 'ts_doy' in ds_dict:
                        #Load surface temperature maps
                        gf.ts_doy = np.ma.array(iolib.ds_getma(ds_dict['ts_doy']), mask=glac_geom_mask)
                        gf.ts_doy.mask = np.ma.mask_or(glac_geom_mask, 
                                                       np.ma.getmask(np.ma.masked_array(gf.ts_doy.data, np.isnan(gf.ts_doy.data))))
                    else:
                        gf.ts_doy = None

                gf.res = geolib.get_res(ds_dict['z1'])

                if verbose:
                    print('Area [km2]:', gf.glac_area / 1e6)
                    print('-------------------------------')
                    
                # Isolate values with positive surface temperatures below mean elevation
                zmean_mask = np.ma.mask_or(glac_geom_mask,  
                                           np.ma.getmask(np.ma.masked_greater(gf.z1, gf.z1.compressed().mean())))
                ts_zmean_mask = np.ma.mask_or(zmean_mask,
                                              np.ma.getmask(np.ma.masked_less(gf.ts, 0)))

                gf.ts_doy.mask = zmean_mask
                gf.ts_year.mask = zmean_mask
                gf.ts_dayfrac.mask = zmean_mask
                doy_list.extend(list(gf.ts_doy.compressed()))
                year_list.extend(list(gf.ts_year.compressed()))
                dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))
                
    
    # Compute statistics
    year_mean_latlon = np.mean(year_list)
    year_std_latlon = np.std(year_list)
    year_med_latlon = malib.fast_median(year_list)
    year_mad_latlon = malib.mad(year_list)
    doy_mean_latlon = np.mean(doy_list)
    doy_std_latlon = np.std(doy_list)
    doy_med_latlon = malib.fast_median(doy_list)
    doy_mad_latlon = malib.mad(doy_list)
    dayfrac_mean_latlon = np.mean(dayfrac_list)
    dayfrac_std_latlon = np.std(dayfrac_list)
    dayfrac_med_latlon = malib.fast_median(dayfrac_list)
    dayfrac_mad_latlon = malib.mad(dayfrac_list)
    
    # Update array
    year_mean[lat_idx,lon_idx] = year_mean_latlon
    year_std[lat_idx,lon_idx] = year_std_latlon
    year_med[lat_idx,lon_idx] = year_med_latlon
    year_mad[lat_idx,lon_idx] = year_mad_latlon
    doy_mean[lat_idx,lon_idx] = doy_mean_latlon
    doy_std[lat_idx,lon_idx] = doy_std_latlon
    doy_med[lat_idx,lon_idx] = doy_med_latlon
    doy_mad[lat_idx,lon_idx] = doy_mad_latlon
    dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
    dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
    dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
    dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon
    
    print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
    print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
    print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
    print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))   
    
    

0 55.25 230.5 5525N-23050E_debris_melt_curve.nc
  processing 01.05754


HACK TO BYPASS VALID AREA


  processing 01.05790
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 243.2 10.2
    doy median +/- mad: 248.8 0.0
  dayfrac mean +/- std: 19.669 0.04
1 55.25 230.75 5525N-23075E_debris_melt_curve.nc
  processing 01.05791
  processing 01.05803
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 237.2 11.8
    doy median +/- mad: 239.0 15.3
  dayfrac mean +/- std: 19.692 0.047
2 55.75 230.25 5575N-23025E_debris_melt_curve.nc
  processing 01.05672
  processing 01.06268
  year mean +/- std: 2013.0 0.1
  doy mean +/- std: 223.7 4.0
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.744 0.013
3 55.75 230.75 5575N-23075E_debris_melt_curve.nc
  processing 01.04425
  year mean +/- std: 2013.0 0.2
  doy mean +/- std: 234.1 12.9
    doy median +/- mad: 226.3 10.0
  dayfrac mean +/- std: 19.696 0.05
4 56.0 229.5 5600N-22950E_debris_melt_curve.nc
  processing 01.04979


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


  processing 01.07101


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


  processing 01.07196


...

  year mean +/- std: 2013.8 1.3
  doy mean +/- std: 206.5 21.1
    doy median +/- mad: 193.9 0.0
  dayfrac mean +/- std: 21.1 0.442
216 63.5 214.5 6350N-21450E_debris_melt_curve.nc
  processing 01.00556


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


  year mean +/- std: 2014.7 0.7
  doy mean +/- std: 175.6 14.1
    doy median +/- mad: 169.9 1.1
  dayfrac mean +/- std: 19.179 4.588
217 63.75 213.0 6375N-21300E_debris_melt_curve.nc
  processing 01.00041


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


  year mean +/- std: 2014.8 0.7
  doy mean +/- std: 175.5 12.8
    doy median +/- mad: 167.9 0.0
  dayfrac mean +/- std: 21.123 0.033


In [8]:
print('DONE')

DONE


In [11]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 35, longitude: 106)
Coordinates:
  * latitude      (latitude) float64 55.25 55.5 55.75 56.0 ... 63.25 63.5 63.75
  * longitude     (longitude) float64 204.5 204.8 205.0 ... 230.2 230.5 230.8
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude

In [9]:
lat_idx = 68
lon_idx = 88
# lat_idx = 37
# lon_idx = 46
print(ds_latlon['latitude'][lat_idx].values, ds_latlon['longitude'][lon_idx].values,
      '\n', ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
      ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

28.0 87.0 
 2015.0001220703125 0.0016057980246841908 229.2476348876953 51.61998748779297


In [10]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/HMA_debris_tsinfo.nc'